<a href="https://colab.research.google.com/github/jorgejaramillo/PythonSEO/blob/master/Quick_Contador_Queries_vs_Titles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





### Resources
You can find example CSV files in the sheet: https://docs.google.com/spreadsheets/d/18e5Xd1l2dHojd91L7e1saY0xGufsIdhPUYe8yDL88Ag/edit?usp=sharing


Créditos: Michael Van Den Reym [Video](https://https://www.youtube.com/watch?v=2gYiWmvBhXE)


Adaptación: Jorge Jaramillo ([jorgejaramillo.com](https://jorgejaramillo.com))

### Step 0: Language settings

In [2]:
langcode="es" #@param {type:"string"}
language="spanish" #@param {type:"string"}

### Step 1 : Import modules

In [ ]:
!pip install stop_words
!pip install python-levenshtein

In [4]:
#Importamos módulos
import pandas as pd 
#input/output modules for uploading and downloading files
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
import io
#install nlp modules not yet available in colab
import re #NLP regex module
import nltk #general NLP module
from nltk.stem.snowball import SnowballStemmer #NLP stemming module
from Levenshtein import * #NLP distance caluclator
from stop_words import get_stop_words #stop words NLP library
nltk.download('punkt') #tokenizer NLP 
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Step 2 : Upload the page title file

In [ ]:
#Step 2: Usamos el id del Spreadsheet idArchivoTitles

gc = gspread.authorize(GoogleCredentials.get_application_default())
idArchivoTitles = 'ID'
worksheet = gc.open_by_key(idArchivoTitles).sheet1
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
dftitles = pd.DataFrame(worksheet.get_all_records())
dftitles.columns=["Page","Title"]
dftitles.head()

### Step 3: Upload second CSV file: the search queries by page 

In [ ]:
#Step 3: Usamos el id del Spreadsheet idArchivoQueries

gc = gspread.authorize(GoogleCredentials.get_application_default())
idArchivoQueries = 'ID'
worksheet = gc.open_by_key(idArchivoQueries).sheet1
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
dfqueries = pd.DataFrame(worksheet.get_all_records())
dfqueries.columns=["Query","Page"]
dfqueries.head()

### Step 4: Combining dataframes

In [ ]:
df = pd.merge(dfqueries,dftitles,how="inner",on='Page')
df.head()

### Step 5: Count number of words

In [ ]:
stop_words = get_stop_words(langcode)

def number_words(sentence):
  sentencewords = nltk.word_tokenize(str(sentence).lower()) 
  not_stop_words=[]
  for word in sentencewords:
    if(word not in stop_words):
        not_stop_words.append(word)
  sentencelength = len(not_stop_words)
  return sentencelength

df["Words in query"]=df.apply(lambda x:number_words(x["Query"]),axis=1)
df["Words in title"]=df.apply(lambda x:number_words(x["Title"]),axis=1)
df.head()

### Step 6: Count number of common words
words of query in title

In [ ]:
def common_words(query,title):
    stemmer  = SnowballStemmer(language, ignore_stopwords=True)
    titlewords = nltk.word_tokenize(str(title).lower())
    stemtitlewords = [stemmer.stem(word) for word in titlewords]
    querywords = nltk.word_tokenize(str(query).lower()) 
    stemquerywords = [stemmer.stem(word) for word in querywords]
    titlenospaces=title.replace(' ','').lower()
    title_only_text = re.sub(r'\W+', '', titlenospaces)
    
    lev_distance=0  
    commonwords=0

    for word in stemquerywords:
      if(word not in stop_words):
          #literal word check
          if(word in title_only_text):
              commonwords=commonwords+1         
          else:
          #check with word with title stemmed and for misspellings
            for word2 in stemtitlewords:
                lev_distance = distance(str(word),str(word2))
                procent_distance = lev_distance/(len(word)+len(word2)/2)
                if procent_distance<0.15:
                    commonwords=commonwords+1
    return commonwords
    
df["Common words"]=df.apply(lambda x:common_words(x["Query"],x["Title"]),axis=1)
df.sample(5)

### Step 7 sort and explore table

In [ ]:
%load_ext google.colab.data_table

print("Possible optimizations")
print("----------------------")

df=df.sort_values(by=['Common words', 'Words in query'], ascending=[True, False])
df

### Additional step: Export and download

In [ ]:
#automatically download
df.to_csv('export.csv')
files.download('export.csv') 